<h2><center>CS5785 AML Milestone</center></h2>
<center>Jinquan Wang (jw2657), Yutong Wang(yw2364), Ling Zhong (lz555) </center>

In [1]:
!pip install nltk==3.5

     |████████████████████████████████| 1.4 MB 12.9 MB/s 
     |████████████████████████████████| 82 kB 3.3 MB/s 
     |████████████████████████████████| 719 kB 54.9 MB/s 
     |████████████████████████████████| 69 kB 19.2 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434674 sha256=ae30520d8010aa3e4b0438c1dceed8eeb9ee2e8cd7b1ff6867739f5838c0574a
  Stored in directory: /home/jovyan/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
! pip install xgboost

You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [11]:
pip install plotly

     |████████████████████████████████| 13.2 MB 18.5 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=5ae3d0a291ab7875ec87bb3bb69fdf42a95cf86dd54d84e0a169c89681d496be
  Stored in directory: /home/jovyan/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import numpy as np
import collections
import re, string
import nltk
import matplotlib.pyplot as plt
import plotly.express as px 
import seaborn as sns
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

ModuleNotFoundError: No module named 'seaborn'

In [6]:
#build the train and test dataset.
data = pd.read_csv('Combined_News_DJIA.csv')
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

In [7]:
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

Providing More details about features: So here we print out the number of samples and the number of features.remember to put them in final write up!!!！
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [8]:
print('{} samples in train set'.format(data.shape[0]))
print('{} columns in train set'.format(data.shape[1]))
print(data.columns)

1989 samples in train set
27 columns in train set
Index(['Date', 'Label', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7',
       'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15',
       'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23',
       'Top24', 'Top25'],
      dtype='object')


In [9]:
print('Response 0: {} ({:.2f}%)'.format(data.Label.value_counts()[0], (data.Label.value_counts()[0]/len(data))*100))
print('Response 1: {} ({:.2f}%)'.format(data.Label.value_counts()[1], (data.Label.value_counts()[1]/len(data))*100))

Response 0: 924 (46.46%)
Response 1: 1065 (53.54%)


In [ ]:
fig, ax1 = plt.subplots(figsize=(4,4))
plot = sns.countplot(train.Response)
plot.set_xticklabels(plot.get_xticklabels())
for p in plot.patches:
    height = p.get_height()
    plot.text(p.get_x()+p.get_width()/2., height + 0.1,height ,ha="center")

In [5]:
print(train.shape)
print(test.shape)

(1611, 27)
(378, 27)


In [6]:

#preprocess the train dataset

# Strip punctuatuion 
def strip_punct(text):
    if not text: return None
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

# tokenize sentences and remove stopwords
# reference: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
# download nltk dataset wordnet and stopwords 


# lemmatize and remove the stopwords
stop_words = set(stopwords.words('english')) 
ps = PorterStemmer() 


def strip_stopword(text, options = "train"):
    filtered_sentence = []
    word_tokens = word_tokenize(text) # tokenize the text into words 
    for w in word_tokens: 
        if w not in stop_words:
            word = ps.stem(w)  #lemmatization 
            filtered_sentence.append(word)
            # if option == "train":
            #     corpus.append(filtered_sentence) 
    # detokenize the sentence after filtering out the stopwords
    filtered_sentence = TreebankWordDetokenizer().detokenize(filtered_sentence) 
    # if options == "train":
    #     corpus.append(filtered_sentence)
    return filtered_sentence

def tokenize(sentence):
    tokens = nltk.word_tokenize(sentence) 
    
    return tokens

def wordLem(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized = ' '.join([lemmatizer.lemmatize(t) for t in tokens])
   
    return lemmatized

In [7]:

train = train.fillna('0')
test = test.fillna('0')


#strip punctuation & Lowercase & strip stopwords for train and test4


for column in train.columns[2:]:
    
    train[column] = train[column].apply(lambda x: strip_punct(x))
    train[column] = [x.lower() for x in train[column]]
    train[column] = train[column].apply(lambda x: strip_stopword(x))
    
    for i in train.index:
        if train.at[i,'Date'] < '2010-07-01':
            train.at[i,column] = train[column][i][1:]
    train[column] = train[column].apply(lambda x:tokenize(x))
    train[column] = train[column].apply(lambda x:wordLem(x))



for column in test.columns[2:]:

    test[column] = test[column].apply(lambda x: strip_punct(x))
    test[column] = [x.lower() for x in test[column]]
    test[column] = test[column].apply(lambda x: strip_stopword(x))
    test[column] = test[column].apply(lambda x:tokenize(x))
    test[column] = test[column].apply(lambda x:wordLem(x))
    # for i in test.index:
    #     test.at[i,column] = test[column][i][1:]


In [8]:
train.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,georgia down two russian warplan countri move ...,break musharraf impeach,russia today column troop roll south ossetia f...,russian tank move toward capit south ossetia r...,afghan child rape impun un offici say sick thr...,150 russian tank enter south ossetia whilst ge...,break georgia invad south ossetia russia warn ...,the enemi combat trial noth sham salim haman s...,...,georgia invad south ossetia russia get involv ...,alqaeda face islamist backlash,condoleezza rice u would act prevent isra stri...,thi busi day european union approv new sanctio...,georgia withdraw 1000 soldier iraq help fight ...,whi pentagon think attack iran bad idea u news...,caucasu crisi georgia invad south ossetia,indian shoe manufactori seri like work,visitor suffer mental ill ban olymp,no help mexico kidnap surg
1,2008-08-11,1,whi wont america nato help u wont help u help ...,bush put foot georgian conflict,jewish georgian minist thank isra train fend r...,georgian armi flee disarray russian advanc gor...,olymp open ceremoni firework fake,what mossad fraudul new zealand passport iraq,russia anger isra militari sale georgia,an american citizen live sossetia blame u geor...,...,israel u behind georgian aggress,do believ tv neither russian georgian much victim,riot still go montreal canada polic murder boy...,china overtak u largest manufactur,war south ossetia pic,isra physician group condemn state tortur,russia beaten unit state head peak oil,perhap question georgia russia conflict,russia much better war,so come trade sex food
2,2008-08-12,0,rememb ador 9yearold sang open ceremoni fake,russia end georgia oper,if sexual harass would child,alqaeda lose support iraq brutal crackdown act...,ceasefir georgia putin outmaneuv west,whi microsoft intel tri kill xo 100 laptop,stratfor russogeorgian war balanc power,im tri get sen whole georgiarussia war vote th...,...,u troop still georgia know georgia first place,whi russia respons georgia right,gorbachev accus u make seriou blunder pursu in...,russia georgia nato cold war two,rememb ador 62yearold led countri war base evi...,war georgia isra connect,all sign point u encourag georgia invad south ...,christoph king argu u nato behind georgian inv...,america new mexico,bbc news asiapacif extinct man climat
3,2008-08-13,0,u refus israel weapon attack iran report,when presid order attack tskhinvali capit sout...,israel clear troop kill reuter cameraman,britain polici tough drug pointless say former...,bodi 14 year old found trunk latest ransom pai...,china move 10 million quak survivor prefab home,bush announc oper get russia grill yeah end well,russian forc sink georgian ship,...,eleph extinct 2020,u humanitarian mission soon georgia russia hit...,georgia ddo came u sourc,russian convoy head georgia violat truce,isra defenc minist u strike iran,gorbachev choic,wit russian forc head toward tbilisi breach ce...,quarter russian blame u conflict poll,georgian presid say u militari take control se...,2006 nobel laureat aleksand solzhenitsyn accus...
4,2008-08-14,1,all expert admit legalis drug,war south osetia 89 pictur made russian soldier,swedish wrestler ara abrahamian throw away med...,russia exagger death toll south ossetia 44 ori...,missil kill 9 insid pakistan may launch cia,rushdi condemn random hous refus publish novel...,poland u agre missl defens deal interest time,will russian conquer tblisi bet serious bet,...,bank analyst forecast georgian crisi 2 day earli,georgia confict could set back russia u relat ...,war caucasu much product american imperi drive...,nonmedia photo south ossetiageorgia conflict,georgian tv report shot russian sniper live br...,saudi arabia mother move block child marriag,taliban wage war humanitarian aid worker,russia world forget georgia territori integr,darfur rebel accus sudan mount major attack,philippin peac advoc say muslim need assur chr...


In [9]:
train_txt = []
for row in range(0, len(train.index)):
    train_txt.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [10]:
test_txt = []
for row in range(0,len(test.index)):
    test_txt.append(' '.join(str(x) for x in test.iloc[row,2:27]))

In [11]:
#Form bag of word and vectorize the train_txt
vectorizer = CountVectorizer(binary=True, min_df=1)

train_content = vectorizer.fit_transform(train_txt)
train_vocab = vectorizer.vocabulary_
test_content = vectorizer.transform(test_txt)

<b>Logistic Regression</b>

In [12]:
logreg=LogisticRegression()
logreg.fit(train_content, train['Label'])
y_pred = logreg.predict(test_content)
y_true = test['Label']

f1_lr = f1_score(y_true, y_pred)
print('f1 score for logistic regression is: ', f1_lr)

f1 score for logistic regression is:  0.5250596658711216


In [13]:
def print_lgreg_top10(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    print('The top ten feature names are:')
    feature_names = vectorizer.get_feature_names()

    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[0])[-10:][::-1]
        print("%s: %s" % (class_label,
              ", ".join(feature_names[j] for j in top10)))



print_lgreg_top10(vectorizer, logreg, logreg.classes_)

The top ten feature names are:
0: el, nigeria, territori, london, kick, scrap, flood, larg, tv, jew
1: el, nigeria, territori, london, kick, scrap, flood, larg, tv, jew


In [14]:

new_vectorizer = TfidfVectorizer( min_df=0.01, max_df=0.99, max_features = 200000, ngram_range = (1, 1))
# new_vectorizer = TfidfVectorizer()
train_new = new_vectorizer.fit_transform(train_txt)
test_new = new_vectorizer.transform(test_txt)

In [15]:
logreg.fit(train_new, train['Label'])
y_pred = logreg.predict(test_new)
y_true = test['Label']
f1_lr2 = f1_score(y_true, y_pred)
print('The f1-score for logistic regression by tf-idf model is: ', f1_lr2)

The f1-score for logistic regression by tf-idf model is:  0.6206896551724138


In [16]:
print_lgreg_top10(new_vectorizer, logreg, logreg.classes_)

The top ten feature names are:
0: nigeria, olymp, scrap, right, gay, rule, mubarak, state, territori, canadian
1: nigeria, olymp, scrap, right, gay, rule, mubarak, state, territori, canadian


## Linear SVM

In [17]:
scores = []
best_score,best_svm = 0, None
c_vals = [0.01,0.1,1.0,10.0,100.0]
y_true = test['Label']
for c in c_vals:
    print("running svm with C = {}".format(c))
    svm = LinearSVC(C=c)
    svm.fit(train_content,train['Label'])
    svm_preds = svm.predict(test_content)
    svm_score = f1_score(y_true , svm_preds)
    print("score for this svm: "+ str(svm_score))
    scores.append(svm_score)
    if svm_score > best_score:
        best_score = svm_score
        best_svm = svm

running svm with C = 0.01
score for this svm: 0.5083135391923991
running svm with C = 0.1
score for this svm: 0.513064133016627
running svm with C = 1.0
score for this svm: 0.5095238095238096
running svm with C = 10.0
score for this svm: 0.5095238095238096
running svm with C = 100.0
score for this svm: 0.5095238095238096


In [18]:
def print_lsvm_top10(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    print('The top ten feature names are:')
    feature_names = vectorizer.get_feature_names()

    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[0])[-10:][::-1]
        print("%s: %s" % (class_label,
              ", ".join(feature_names[j] for j in top10)))


In [19]:
print_lsvm_top10(vectorizer,best_svm, best_svm.classes_ )

The top ten feature names are:
0: el, nigeria, territori, london, kick, scrap, flood, 45, worri, larg
1: el, nigeria, territori, london, kick, scrap, flood, 45, worri, larg


As we can tell, even though we switch to another model, the F1 score does not vary too much. Thus,
we want to utilize an advanced vectorizer.
The goal of using tf-idf instead of the raw frequencies of occurrence of a token in a given document 
is to scale down the impact of tokens that occur very frequently in a given corpus and that are hence 
empirically less informative than features that occur in a small fraction of the training corpus.



In [20]:
new_vectorizer = TfidfVectorizer( min_df=0.001, max_df=0.99, max_features = 200000, ngram_range = (1, 1))
train_new = new_vectorizer.fit_transform(train_txt)
test_new = new_vectorizer.transform(test_txt)

In [21]:
scores = []
best_score,best_svm = 0, None
c_vals = [0.01,0.1,1.0,10.0,100.0]
y_true = test['Label']
for c in c_vals:
    print("running svm with C = {}".format(c))
    svm = LinearSVC(C=c)
    svm.fit(train_new,train['Label'])
    svm_preds = svm.predict(test_new)
    svm_score = f1_score(y_true , svm_preds)
    print("score for this svm: "+ str(svm_score))
    scores.append(svm_score)
    if svm_score > best_score:
        best_score = svm_score
        best_svm = svm

running svm with C = 0.01
score for this svm: 0.6736842105263158
running svm with C = 0.1
score for this svm: 0.6451612903225806
running svm with C = 1.0
score for this svm: 0.5688487584650113
running svm with C = 10.0
score for this svm: 0.5668202764976958
running svm with C = 100.0
score for this svm: 0.5635103926096998


In [22]:



print_lsvm_top10(new_vectorizer, best_svm, best_svm.classes_)

The top ten feature names are:
0: nigeria, mubarak, olymp, gay, right, rule, state, scrap, canadian, forc
1: nigeria, mubarak, olymp, gay, right, rule, state, scrap, canadian, forc


## NonLinear-SVM

In [23]:
scores = []
best_score,best_svm = 0, None
c_vals = [0.01,0.1,1.0,10.0,100.0]
y_true = test['Label']
for c in c_vals:
    print("running svm with C = {}".format(c))
    svm = SVC(C=c, kernel = "rbf")
    svm.fit(train_content,train['Label'])
    svm_preds = svm.predict(test_content)
    svm_score = f1_score( y_true, svm_preds)
    print("score for this svm: "+ str(svm_score))
    scores.append(svm_score)
    if svm_score > best_score:
        best_score = svm_score
        best_svm = svm

running svm with C = 0.01
score for this svm: 0.6736842105263158
running svm with C = 0.1
score for this svm: 0.6736842105263158
running svm with C = 1.0
score for this svm: 0.6617915904936016
running svm with C = 10.0
score for this svm: 0.5631929046563193
running svm with C = 100.0
score for this svm: 0.5631929046563193


In [24]:
scores = []
best_score,best_svm = 0, None
c_vals = [0.01,0.1,1.0,10.0,100.0]
y_true = test['Label']
for c in c_vals:
    print("running svm with C = {}".format(c))
    svm = SVC(C=c,kernel = "rbf")
    svm.fit(train_new,train['Label'])
    svm_preds = svm.predict(test_new)
    svm_score = f1_score(y_true , svm_preds)
    print("score for this svm: "+ str(svm_score))
    scores.append(svm_score)
    if svm_score > best_score:
        best_score = svm_score
        best_svm = svm



running svm with C = 0.01
score for this svm: 0.6736842105263158
running svm with C = 0.1
score for this svm: 0.6736842105263158
running svm with C = 1.0
score for this svm: 0.6607460035523979
running svm with C = 10.0
score for this svm: 0.6021052631578948
running svm with C = 100.0
score for this svm: 0.6021052631578948


# Random Forest

In [25]:

new_vectorizer = TfidfVectorizer( min_df=0.01, max_df=0.99, max_features = 200000, ngram_range = (1, 1))
# new_vectorizer = TfidfVectorizer()
train_new = new_vectorizer.fit_transform(train_txt)
test_new = new_vectorizer.transform(test_txt)

In [26]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=12, ccp_alpha=0.5)
rfc_trained = rfc.fit(train_new, train['Label'])
y_pred = rfc_trained.predict(test_new)

y_true = test['Label']
f1_rf = f1_score(y_true, y_pred)
print('F1 score generated by random forest classifier is: ', f1_rf)

F1 score generated by random forest classifier is:  0.6736842105263158


In [27]:
def print_rfc_top10(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    print('The top ten feature names are:')
    feature_names = vectorizer.get_feature_names()

    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.feature_importances_, -1)[-10:][::-1]
        print("%s: %s" % (class_label,
              ", ".join(feature_names[j] for j in top10)))



print_rfc_top10(new_vectorizer, rfc_trained, rfc_trained.classes_)

The top ten feature names are:
0: zoo, evacu, exhibit, exercis, execut, excus, exclus, exchang, excess, except
1: zoo, evacu, exhibit, exercis, execut, excus, exclus, exchang, excess, except


# XGBoosting

In [28]:
model_xgb = XGBClassifier(max_depth=12, n_estimators=100)
model_xgb.fit(train_new, train['Label'])
y_pred = model_xgb.predict(test_new)
y_true = test['Label']

f1_xgb = f1_score(y_true, y_pred)
print('F1 score generated by XGBoost classifer is: ', f1_xgb)

F1 score generated by XGBoost classifer is:  0.5772727272727273


In [29]:
def print_xgb_top10(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    print('The top ten feature names are:')
    feature_names = vectorizer.get_feature_names()

    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.feature_importances_, -1)[-10:][::-1]
        print("%s: %s" % (class_label,
              ", ".join(feature_names[j] for j in top10)))



print_xgb_top10(new_vectorizer, model_xgb, model_xgb.classes_)

The top ten feature names are:
0: abl, beach, abdullah, photograph, brazilian, eye, graphic, infect, ocean, aim
1: abl, beach, abdullah, photograph, brazilian, eye, graphic, infect, ocean, aim
